# AgentCore Runtime에서 Runtime Context와 Session 관리 이해하기

## 개요

이 튜토리얼에서는 Amazon Bedrock AgentCore Runtime에서 runtime context와 session 관리를 이해하고 작업하는 방법을 배웁니다. 이 예제는 AgentCore Runtime이 session을 처리하고, 여러 호출에 걸쳐 context를 유지하며, agent가 context 객체를 통해 runtime 정보에 액세스하는 방법을 보여줍니다.

Amazon Bedrock AgentCore Runtime은 각 사용자 상호작용에 대해 격리된 session을 제공하여, agent가 여러 호출에 걸쳐 context와 상태를 유지하면서 서로 다른 사용자 간의 완전한 보안 격리를 보장합니다.

### 튜토리얼 세부 정보

|정보| 세부사항|
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형       | Context 및 Session 관리|
| Agent 유형          | Single         |
| Agentic Framework   | Strands Agents |
| LLM model           | Anthropic Claude Haiku 4.5 |
| 튜토리얼 구성요소 | Runtime Context, Session Management, AgentCore Runtime, Strands Agent and Amazon Bedrock Model |
| 튜토리얼 분야   | 범용                                                                   |
| 예제 복잡도  | 중급                                                                     |
| 사용된 SDK            | Amazon BedrockAgentCore Python SDK and boto3|

### 튜토리얼 아키텍처

이 튜토리얼에서는 Amazon Bedrock AgentCore Runtime이 session을 관리하고 agent에게 context를 제공하는 방법을 살펴봅니다. 다음을 시연합니다:

1. **Session 연속성**: 동일한 session ID가 여러 호출에 걸쳐 context를 유지하는 방법
2. **Context 객체**: agent가 context 매개변수를 통해 runtime 정보에 액세스하는 방법
3. **Session 격리**: 서로 다른 session ID가 완전히 격리된 환경을 생성하는 방법
4. **Payload 유연성**: payload를 통해 agent에게 커스텀 데이터를 전달하는 방법

시연 목적으로 이러한 session 관리 기능을 보여주는 Strands Agent를 사용합니다.

    
<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="60%"/>
</div>

### 튜토리얼 주요 기능

* **Session 기반 Context 관리**: AgentCore Runtime이 session 내에서 context를 유지하는 방법 이해
* **Runtime Session 수명주기**: session 생성, 유지 및 종료에 대해 학습
* **Context 객체 액세스**: context 매개변수를 통해 session ID와 같은 runtime 정보에 액세스
* **Session 격리**: 서로 다른 session이 완전한 격리를 제공하는 방법 시연
* **Payload 처리**: 커스텀 payload 구조를 통한 유연한 데이터 전달
* **호출 간 상태**: 동일한 session 내에서 여러 호출에 걸쳐 agent 상태 유지

## 사전 요구사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker running

## Amazon Bedrock AgentCore Runtime Session 이해하기

코드를 살펴보기 전에 Amazon Bedrock AgentCore Runtime이 session을 관리하는 방법을 이해하는 것이 중요합니다:

### Session 격리 및 보안

AgentCore Runtime은 전용 microVM을 통해 **완전한 session 격리**를 제공합니다:

- **전용 리소스**: 각 session은 격리된 CPU, 메모리 및 파일시스템을 가진 자체 microVM에서 실행됩니다
- **보안 경계**: 사용자 session 간의 완전한 분리로 데이터 오염을 방지합니다
- **결정적 정리**: session 완료 후 microVM이 종료되고 메모리가 정리됩니다

### Session 수명주기

AgentCore Runtime의 session은 특정 수명주기를 따릅니다:

1. **생성**: 첫 번째 호출 시 고유한 `runtimeSessionId`로 새 session이 생성됩니다
2. **활성 상태**: Session이 요청을 처리하고 context를 유지합니다
3. **유휴 상태**: Session이 context를 보존하면서 다음 호출을 기다립니다
4. **종료**: 다음과 같은 이유로 session이 종료됩니다:
   - 비활성 (15분)
   - 최대 수명 (8시간)
   - 상태 확인 실패

### Context 지속성

session 내에서 AgentCore Runtime은 다음을 유지합니다:
- **대화 기록**: 이전 상호작용 및 응답
- **애플리케이션 상태**: 실행 중 생성된 변수 및 객체
- **파일 시스템**: session 중 생성되거나 수정된 모든 파일
- **환경 변수**: 커스텀 설정 및 구성

### Session 관리 모범 사례

- **고유한 Session ID**: 각 사용자 또는 대화에 대해 고유한 session ID를 생성합니다
- **Context 재사용**: context를 유지하기 위해 관련 호출에 동일한 session ID를 사용합니다
- **Session 경계**: 서로 다른 사용자 또는 관련 없는 대화에 대해 다른 session ID를 사용합니다
- **임시 특성**: 영구 데이터 저장을 위해 session에 의존하지 마십시오 (지속성을 위해 AgentCore Memory 사용)

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## AgentCore Runtime에 배포하기 위한 agent 준비

이제 session 관리 및 context 처리를 시연하기 위해 agent를 AgentCore Runtime에 배포해 보겠습니다. 우리의 agent는 다음 방법을 보여줍니다:

1. **Runtime Context 액세스**: `context` 매개변수를 사용하여 session 정보 가져오기
2. **커스텀 Payload 처리**: payload를 통해 전달된 구조화된 데이터 처리
3. **Session 상태 유지**: session 내에서 사용자 상호작용 추적
4. **Session 경계 시연**: 서로 다른 session이 격리되는 방법 표시

### Context 객체 이해하기

AgentCore Runtime의 `context` 객체는 현재 실행 환경에 대한 유용한 정보를 제공합니다:

- **session_id**: 현재 runtime session 식별자
- **Runtime Metadata**: runtime 환경에 대한 정보
- **실행 세부정보**: 현재 호출에 대한 context

### Context 처리를 포함한 Strands Agent

session 관리 및 context 처리를 시연하는 구현을 살펴보겠습니다:

In [ ]:
%%writefile strands_claude_context.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel
import asyncio
from datetime import datetime

# AgentCore Runtime 애플리케이션 인스턴스 생성
app = BedrockAgentCoreApp()

# @tool 데코레이터로 agent가 호출할 수 있는 함수 정의
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"

@tool
def get_time():
    """ Get current time """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

model_id = "global.anthropic.claude-haiku-4-5-20251001-v1:0"
# Bedrock 모델 초기화
model = BedrockModel(
    model_id=model_id,
)
# Agent 생성 시 model, tools, system_prompt 설정
agent = Agent(
    model=model,
    tools=[
        calculator, weather, get_time
    ],
    system_prompt="""
    You're a helpful assistant. You can do simple math calculations, 
    tell the weather, and provide the current time.
    Always start by acknowledging the user's name 
    """
)

def get_user_name(user_id):
    users = {
        "1": "Maira",
        "2": "Mani",
        "3": "Mark",
        "4": "Ishan",
        "5": "Dhawal"
    }
    return users[user_id]
    
# @app.entrypoint: AgentCore Runtime이 호출하는 진입점 함수
# payload: 클라이언트가 전달한 데이터, context: Runtime이 제공하는 session 정보
@app.entrypoint
def strands_agent_bedrock_handling_context(payload, context):
    """
    AgentCore Runtime entrypoint that demonstrates context handling and session management.
    
    Args:
        payload: The input payload containing user data and request information
        context: The runtime context object containing session and execution information
    
    Returns:
        str: The agent's response incorporating context information
    """
    # payload에서 사용자 입력 추출
    user_input = payload.get("prompt")
    user_id = payload.get("user_id")
    user_name = get_user_name(user_id)
    
    # context.session_id로 Runtime이 관리하는 session 정보 접근
    print("=== Runtime Context Information ===")
    print("User id:", user_id)
    print("User Name:", user_name)
    print("User input:", user_input)
    print("Runtime Session ID:", context.session_id)
    print("Context Object Type:", type(context))
    print("=== End Context Information ===")
    
    # session ID를 포함한 개인화된 prompt 생성
    prompt = f"""My name is {user_name}. Here is my request: {user_input}
    
    Additional context: This is session {context.session_id}. 
    Please acknowledge my name and provide assistance."""
    
    # agent 호출 및 응답 추출
    response = agent(prompt)
    # response.message는 중첩된 구조이므로 text 부분만 반환
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

## AgentCore Runtime의 Session 관리 이해하기

위의 코드는 AgentCore Runtime이 session을 관리하고 agent에게 context를 제공하는 방법에 대한 몇 가지 주요 개념을 보여줍니다:

### Context 객체 구조

entrypoint 함수의 `context` 매개변수는 runtime 정보에 대한 액세스를 제공합니다:

```python
@app.entrypoint
def strands_agent_bedrock_handling_context(payload, context):
    # session 정보 액세스
    session_id = context.session_id
    # agent 로직에서 context 정보 사용
```

### Session 연속성의 이점

단일 session 내에서 AgentCore Runtime은 다음을 제공합니다:

1. **지속적인 환경**: 변수와 상태가 호출 간에 유지됩니다
2. **Context 보존**: agent가 이전 상호작용을 참조할 수 있습니다
3. **리소스 재사용**: 초기화된 model과 tool이 로드된 상태로 유지됩니다
4. **성능 이점**: 후속 호출에 대한 콜드 스타트 시간 감소

### Session 격리 보장

AgentCore Runtime은 session 간의 완전한 격리를 보장합니다:

- **보안**: 각 session은 격리된 리소스를 가진 자체 microVM에서 실행됩니다
- **프라이버시**: 서로 다른 사용자 session 간에 데이터 유출이 없습니다
- **안정성**: 한 session의 문제가 다른 session에 영향을 주지 않습니다
- **정리**: session 종료 후 완전한 메모리 정리

### Payload 유연성

`payload` 매개변수는 유연한 데이터 전달을 허용합니다:

```python
# 예제 payload 구조
payload = {
    "prompt": "User's question",
    "user_id": "1",
    "preferences": {...},
    "context_data": {...}
}
```

이를 통해 runtime이 제공하는 session context를 유지하면서 클라이언트와 agent 간의 풍부하고 구조화된 통신이 가능합니다.

### AgentCore Runtime deployment 구성

다음으로 starter toolkit을 사용하여 entrypoint, 방금 생성한 실행 역할 및 requirements 파일로 AgentCore Runtime deployment를 구성합니다. 또한 시작 시 Amazon ECR repository를 자동으로 생성하도록 starter kit을 구성합니다.

구성 단계에서 애플리케이션 코드를 기반으로 docker 파일이 생성됩니다

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

# AgentCore Runtime 인스턴스 생성
agentcore_runtime = Runtime()

# Runtime 구성: entrypoint 파일, IAM role, ECR repository 자동 생성 설정
response = agentcore_runtime.configure(
    entrypoint="strands_claude_context.py",
    auto_create_execution_role=True,  # IAM 실행 역할 자동 생성
    auto_create_ecr=True,  # ECR repository 자동 생성
    requirements_file="requirements.txt",
    region=region,
    agent_name="strands_claude_context"
)

### AgentCore Runtime에 context-aware agent 시작하기

이제 docker 파일을 얻었으므로 context-aware agent를 AgentCore Runtime에 시작해 보겠습니다. 이렇게 하면 Amazon ECR repository와 AgentCore Runtime이 생성됩니다.

우리의 agent는 AgentCore Runtime이 session을 관리하고 agent에게 context 정보를 제공하는 방법을 보여줍니다.

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()

### AgentCore Runtime 상태 확인
이제 AgentCore Runtime을 배포했으므로 배포 상태를 확인해 보겠습니다

In [ ]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
# Runtime이 최종 상태가 될 때까지 polling
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

## Session 관리 및 Context 처리 시연

이제 다양한 시나리오를 테스트하여 AgentCore Runtime의 주요 session 관리 기능을 시연해 보겠습니다:

### 시나리오 1: Session 연속성
여러 호출에 동일한 session ID를 사용하여 context가 유지되는 방법을 보여줍니다.

### 시나리오 2: Session 격리
서로 다른 session ID를 사용하여 session 간의 완전한 격리를 시연합니다.

### 시나리오 3: Context 정보 액세스
agent가 runtime context 정보에 액세스하는 방법을 보여줍니다.

<div style="text-align:left">
    <img src="images/invoke.png" width="85%"/>
</div>

In [ ]:
import uuid
import json
from IPython.display import Markdown, display

# Session 연속성 시연을 위한 고유 session ID 생성
session_id = uuid.uuid4()
print(f"📋 Starting Session 1: {session_id}")
print(f"👤 User: Maira (ID: 1)")
print(f"❓ First question about weather\n")

# 동일한 session_id로 호출하면 context가 유지됨
invoke_response = agentcore_runtime.invoke({
    "prompt": "How is the weather outside?",
    "user_id": "1"
}, session_id=str(session_id))

response_data = invoke_response['response'][0]
display(Markdown(response_data))

In [ ]:
# Continue with the same session ID to demonstrate session continuity
print(f"🔄 Continuing Session 1: {session_id}")
print(f"👤 Same user: Maira (ID: 1)")
print(f"❓ Follow-up question about math\n")

# 동일한 session_id 사용 - agent가 이전 대화를 기억함
invoke_response = agentcore_runtime.invoke({
    "prompt": "How much is 2X5?",
    "user_id": "1"
}, session_id=str(session_id))

response_data = invoke_response['response'][0]
display(Markdown(response_data))

In [ ]:
# Continue with the same session ID - notice how the agent remembers the previous calculation
print(f"🔄 Continuing Session 1: {session_id}")
print(f"👤 Same user: Maira (ID: 1)")
print(f"❓ Building on previous answer - demonstrates context continuity\n")

# "that"이 이전 계산 결과(10)를 참조 - session context 유지 확인
invoke_response = agentcore_runtime.invoke({
    "prompt": "and that plus 34?",
    "user_id": "1"
}, session_id=str(session_id))

response_data = invoke_response['response'][0]
display(Markdown(response_data))

In [ ]:
# NEW SESSION - Demonstrate session isolation
# Create a completely new session ID to show that context is lost
new_session_id = uuid.uuid4()
print(f"🆕 Starting NEW Session 2: {new_session_id}")
print(f"👤 Same user: Maira (ID: 1)")
print(f"❓ Attempting to reference previous calculation - should fail due to session isolation\n")

# 새로운 session_id 사용 - 이전 session의 context에 접근 불가
invoke_response = agentcore_runtime.invoke({
    "prompt": "And plus 10?",
    "user_id": "1"
}, session_id=str(new_session_id))

response_data = invoke_response['response'][0]
display(Markdown(response_data))

In [ ]:
# NEW SESSION AND USER - Demonstrate complete isolation
different_user_session = uuid.uuid4()
print(f"🆕 Starting Session 3: {different_user_session}")
print(f"👤 Different user: Mani (ID: 2)")
print(f"❓ Same question as first user - demonstrates user isolation\n")

# 다른 사용자와 새 session - 완전히 격리된 환경
invoke_response = agentcore_runtime.invoke({
    "prompt": "How is the weather?",
    "user_id": "2"
}, session_id=str(different_user_session))

response_data = invoke_response['response'][0]
display(Markdown(response_data))

## Session 관리 결과 이해하기

위의 시연은 AgentCore Runtime의 session 관리의 몇 가지 주요 측면을 보여줍니다:

### 1. Session 연속성 (Session 1)
- **첫 번째 호출**: Agent가 날씨 질문에 응답하고 사용자 이름을 인식합니다
- **두 번째 호출**: Agent가 계산을 수행합니다 (2×5=10)
- **세 번째 호출**: Agent가 이전 결과를 참조합니다 ("that plus 34" = 44)

**주요 학습 내용**: agent는 동일한 session 내에서 여러 호출에 걸쳐 context를 유지하여 이전 상호작용의 계산 결과를 기억했습니다.

### 2. Session 격리 (Session 2)
- **새 session ID**: 완전히 새로운 session을 생성했습니다
- **동일한 사용자**: 동일한 사용자 ID를 사용했지만 다른 session입니다
- **Context 손실**: Agent가 이전 계산을 참조할 수 없습니다

**주요 학습 내용**: 동일한 사용자라도 새 session은 이전 context에 액세스할 수 없는 완전히 격리된 환경을 생성합니다.

### 3. 사용자 및 Session 격리 (Session 3)
- **다른 사용자**: Maira 대신 Mani
- **새 session**: 이전 session과 완전히 격리됨
- **새로운 context**: Agent가 깨끗한 상태로 시작합니다

**주요 학습 내용**: 각 session은 완전한 격리를 제공하여 서로 다른 사용자와 상호작용 간의 프라이버시와 보안을 보장합니다.

### 4. Context 객체 사용
모든 호출에서 agent는:
- `context.session_id`를 통해 runtime context에 액세스했습니다
- 커스텀 payload 데이터(`user_id`, `prompt`)를 처리했습니다
- 로깅 및 디버깅 정보를 유지했습니다

**주요 학습 내용**: context 객체는 agent가 향상된 기능과 디버깅에 사용할 수 있는 유용한 runtime 정보를 제공합니다.

### 시연된 Session 관리 모범 사례

1. 대화 연속성을 위해 **일관된 session ID 사용**
2. 서로 다른 사용자 또는 대화에 대해 **고유한 session ID 생성**
3. 향상된 agent 동작을 위해 **context 정보 활용**
4. **session 경계를 고려한 설계** - session 간 지속성을 가정하지 마십시오
5. session이 변경되거나 만료될 때 **우아한 context 손실 처리**

## 정리 (선택사항)

이제 생성된 AgentCore Runtime을 정리해 보겠습니다

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
import boto3

agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
)

# AgentCore Runtime 삭제
runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
)

# ECR repository 삭제 (force=True로 이미지 포함 전체 삭제)
response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# 축하합니다!

Amazon Bedrock AgentCore Runtime을 사용하여 session 관리 및 context 처리를 성공적으로 구현하고 테스트했습니다! 

## 학습한 내용:

### Session 관리 기초
* **Session 연속성**: 동일한 session ID가 여러 호출에 걸쳐 context를 유지하는 방법
* **Session 격리**: 서로 다른 session ID가 완전히 격리된 환경을 생성하는 방법
* **Context 보존**: agent가 상태를 유지하고 이전 상호작용을 참조하는 방법
* **보안 경계**: AgentCore Runtime이 사용자 간의 완전한 격리를 보장하는 방법

### Runtime Context 처리
* **Context 객체 액세스**: `context` 매개변수를 통해 runtime 정보에 액세스하는 방법
* **Session 정보**: agent 로직에서 session ID를 검색하고 사용하는 방법
* **Payload 처리**: 커스텀 payload를 통해 전달된 구조화된 데이터를 처리하는 방법
* **Runtime Metadata**: agent가 실행 환경 정보에 액세스하는 방법

### AgentCore Runtime 아키텍처
* **MicroVM 격리**: 각 session은 자체 격리된 microVM에서 실행됩니다
* **리소스 관리**: session당 전용 CPU, 메모리 및 파일시스템
* **보안 모델**: session 종료 후 완전한 메모리 정리
* **수명주기 관리**: Session 상태 (활성, 유휴, 종료) 및 타임아웃

### 모범 사례 구현
* **Session ID 생성**: 서로 다른 대화에 대한 고유 식별자 생성
* **Context 활용**: 향상된 agent 동작을 위한 runtime context 활용
* **상태 관리**: 임시 상태와 지속적 상태 이해
* **오류 처리**: context 손실 및 session 경계의 우아한 처리